In [14]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import scipy.optimize as opt
from LinApp_FindSS import LinApp_FindSS
from LinApp_Deriv import LinApp_Deriv
from LinApp_Solve import LinApp_Solve

Exercise 1

$F = \frac{\alpha^{2}\beta}{\alpha\beta ^{\frac{\alpha}{1 - \alpha}} - \alpha\beta^{\frac{1}{1 - \alpha}}}$

$G = -\frac{\alpha^{3}\beta (1 + \beta)}{\alpha\beta ^{\frac{\alpha}{1 - \alpha}} - \alpha\beta^{\frac{1}{1 - \alpha}}}$

$H =  \frac{\alpha^{4}\beta^{2}}{\alpha\beta ^{\frac{\alpha}{1 - \alpha}} - \alpha\beta^{\frac{1}{1 - \alpha}}}$



In [15]:
# problem 4
# We will use the same structure as Dr. Phillips
def Modeldefs(Xp, X, Y, Z, params):
    '''
    This function takes vectors of endogenous and exogenous state variables
    along with a vector of 'jump' variables and returns explicitly defined
    values for consumption, gdp, wages, real interest rates, and transfers
    
    Inputs are:
        Xp: value of capital in next period
        X: value of capital this period
        Y: value of labor this period
        Z: value of productivity this period
        params: list of parameter values
    
    Output are:
        Y: GDP
        w: wage rate
        r: rental rate on capital
        T: transfer payments
        c: consumption
        u: utiity
    '''
    
    # unpack input vectors
    kp = Xp
    k = X
    ell = Y
    z = Z
    
    # find definintion values
    Y = k**alpha*(np.exp(z)*ell)**(1-alpha)
    w = (1-alpha)*Y/ell
    r = alpha*Y/k
    T = tau*(w*ell + (r - delta)*k)
    c = (w*ell + (r - delta)*k)  + k - kp
    u = c**(1-gamma)/(1-gamma) + a*((1 - ell)**(1 - xi) - 1)/(1 - xi) 
    
    return Y, w, r, T, c, u

In [16]:
def Modeldyn(theta0, params):
    '''
    This function takes vectors of endogenous and exogenous state variables
    along with a vector of 'jump' variables and returns values from the
    characterizing Euler equations.
    
    Inputs are:
        theta: a vector containng (Xpp, Xp, X, Yp, Y, Zp, Z) where:
            Xpp: value of capital in two periods
            Xp: value of capital in next period
            X: value of capital this period
            Yp: value of labor in next period
            Y: value of labor this period
            Zp: value of productivity in next period
            Z: value of productivity this period
        params: list of parameter values
    
    Output are:
        Euler: a vector of Euler equations written so that they are zero at the
            steady state values of X, Y & Z.  This is a 2x1 numpy array. 
    '''
    
    # unpack theat0
    (Xpp, Xp, X, Yp, Y, Zp, Z) = theta0
    alpha, beta, gamma, delta, a, xi, rho, tau = params
    ell = Y
    # find definitions for now and next period
    Y, w, r, T, c, u = Modeldefs(Xp, X, Y, Z, params)
    Yp, wp, rp, Tp, cp, up = Modeldefs(Xpp, Xp, Yp, Zp, params)
    
    # Evaluate Euler equations
    
    E1 = (c**(-gamma)*w*(1- tau))-(a*(1 - ell)**(-xi)) 
    E2 = (c**(-gamma)) - (beta*cp**(-gamma)*(1 + (rp - delta)*(1 - tau))) 
    
    
    return np.array([E1, E2])

In [17]:
# set parameter values
alpha = .40
beta = .98
gamma = 2.5
delta = .10
a = .5
xi = 1.5
rho = .9
tau = .05

# make parameter list to pass to functions
params = (alpha, beta, gamma, delta, a, xi, rho, tau)

# set LinApp parameters
Zbar = np.array([0.])
nx = 1
ny = 1
nz = 1
logX = 0
Sylv = 0

In [18]:
# take a guess for steady state values of k and ell
guessXY = np.array([0.1, .25])

# find the steady state values using LinApp_FindSS
XYbar = LinApp_FindSS(Modeldyn, params, guessXY, Zbar, nx, ny)
(k_bar, ell_bar) = XYbar
print ('XYbar: ', XYbar)

# set up steady state input vector
theta0 = np.array([k_bar, k_bar, k_bar, ell_bar, ell_bar, 0., 0.])

# derive the steady state value of other variables
Y_bar, w_bar, r_bar, T_bar, c_bar, u_bar = Modeldefs(k_bar, k_bar, ell_bar, 0, params)
i_bar = delta*k_bar
# check SS solution
check = Modeldyn(theta0, params)
print ('check: ', check)
if np.max(np.abs(check)) > 1.E-6:
    print ('Have NOT found steady state')
print("\noutput:", Y_bar)
print("\nwage:", w_bar)
print("\ninterest rate:", r_bar)
print("\nGovernment transfer:", T_bar)
print("\nConsumption:", c_bar)
print("\nUtility:", u_bar)
print("\nInvestment:", i_bar)



XYbar:  [ 4.22522903  0.57979145]
check:  [  1.95399252e-14  -6.66133815e-15]

output: 1.28322610883

wage: 1.32795276835

interest rate: 0.121482277121

Government transfer: 0.0430351603077

Consumption: 0.860703206154

Utility: -1.37753930478

Investment: 0.422522902678


In [19]:
# problem 5
eps = 1e-4
var_list = np.array([k_bar, ell_bar, Y_bar, w_bar, r_bar, T_bar, c_bar, i_bar])
par_der = np.empty([len(var_list), len(params)])
for iP, param in enumerate(params):
    params_new = params
    lst = list(params_new)
    lst[iP] *= (1 + eps)
    params_new = tuple(lst)
    XYbar_new = LinApp_FindSS(Modeldyn, params_new, guessXY, Zbar, nx, ny)
    (k_bar_new, ell_bar_new) = XYbar_new
    Y_bar_new, w_bar_new, r_bar_new, T_bar_new, c_bar_new, u_bar_new = Modeldefs(k_bar_new, k_bar_new, ell_bar_new, 0, params_new)
    i_bar_new = delta*k_bar_new
    var_list_new = np.array([k_bar_new, ell_bar_new, Y_bar_new, w_bar_new, r_bar_new, T_bar_new, c_bar_new, i_bar_new])
    par_der[:, iP] = (np.array(var_list_new) - np.array(var_list))/(eps*param)
    
print(par_der)





[[  0.00000000e+00   6.55092708e+01   1.38691596e-01  -5.96901883e+01
   -1.84916383e+00  -8.01620336e-01   0.00000000e+00  -2.32323173e+00]
 [  0.00000000e+00   2.61165854e-01   1.90314422e-02  -2.37267395e-01
   -2.53744679e-01  -1.09999391e-01   0.00000000e+00  -1.38924066e-01]
 [  0.00000000e+00   8.30161430e+00   4.21214272e-02  -7.56666969e+00
   -5.61601581e-01  -2.43456660e-01   0.00000000e+00  -4.66716153e-01]
 [  0.00000000e+00   7.99243948e+00  -8.88178420e-12  -7.28699809e+00
    6.66133815e-11   2.66453526e-11   0.00000000e+00  -1.64792371e-01]
 [  0.00000000e+00  -1.09592495e+00   1.22124533e-12   1.00000000e+00
   -1.05471187e-11  -3.60822483e-12   0.00000000e+00   2.26130422e-02]
 [  0.00000000e+00   8.75343610e-02   1.41261338e-03  -7.98825428e-02
   -1.88342599e-02  -8.16473130e-03   0.00000000e+00  -1.17196490e-02]
 [  0.00000000e+00   1.75068722e+00   2.82522676e-02  -1.59765086e+00
   -3.76685199e-01  -1.63294626e-01   0.00000000e+00  -2.34392980e-01]
 [  0.0000000

In [23]:
# find the derivatives matrices
nx_new = 2
ny_new = 0

#theta0 = np.array([k_bar, k_bar, k_bar, ell_bar, ell_bar, Zbar, Zbar])


def Modeldyn2(theta0, params):
    '''
    This function takes vectors of endogenous and exogenous state variables
    along with a vector of 'jump' variables and returns values from the
    characterizing Euler equations.
    
    Inputs are:
        theta: a vector containng (Xpp, Xp, X, Yp, Y, Zp, Z) where:
            Xpp: value of capital in two periods
            Xp: value of capital in next period
            X: value of capital this period
            Yp: value of labor in next period
            Y: value of labor this period
            Zp: value of productivity in next period
            Z: value of productivity this period
        params: list of parameter values
    
    Output are:
        Euler: a vector of Euler equations written so that they are zero at the
            steady state values of X, Y & Z.  This is a 2x1 numpy array. 
    '''
    
    # unpack theat0
    (Xpp, Yp, Xp, Y, X, Ym, Zp, Z) = theta0
    #alpha, beta, gamma, delta, a, xi, rho, tau = params
    
    ell = Y
    # find definitions for now and next period
    Y, w, r, T, c, u = Modeldefs(Xp, X, Y, Z, params)
    Yp, wp, rp, Tp, cp, up = Modeldefs(Xpp, Xp, Yp, Zp, params)
    
    # Evaluate Euler equations
    
    E1 = (c**(-gamma)*w*(1- tau))/(a*(1 - ell)**(-xi)) -1
    E2 = (c**(-gamma))/(beta*cp**(-gamma)*(1 + (rp - delta)*(1 - tau))) - 1
    
    
    return np.array([E1, E2])

theta0_new = np.array([k_bar, ell_bar, k_bar, ell_bar, k_bar, ell_bar, 0., 0.])

#theta0 = np.array([k_bar, k_bar, k_bar, ell_bar, ell_bar, Zbar, Zbar])
[AA, BB, CC, DD, FF, GG, HH, JJ, KK, LL, MM, WW, TT] = \
    LinApp_Deriv(Modeldyn2, params, theta0_new, nx_new, ny_new, nz, logX = 1)
print('FF: ', FF)
print('GG: ', GG)
print('HH: ', HH)
print('LL: ', LL)
print('MM: ', MM)

# set value for NN    
NN = rho
    
# find the policy and jump function coefficients
PP, QQ, UU, RR, SS, VV = \
    LinApp_Solve(AA,BB,CC,DD,FF,GG,HH,JJ,KK,LL,MM,WW,TT,NN,Zbar,Sylv)
print ('PP:',PP)
print ('QQ:', QQ)
print ('RR:',RR)
print ('SS:', SS)
print('NN', NN)

FF:  [[  0.           0.        ]
 [-12.27260743   2.16849645]]
GG:  [[ 12.27260744  -4.70601255]
 [ 24.87671844  -2.23635645]]
HH:  [[-12.13625099   0.        ]
 [-12.53625099   0.        ]]
LL:  [[ 0.        ]
 [ 2.16849645]]
MM:  [[-1.63635645]
 [-2.23635645]]
Sylvester equation solver condition is not satisfied; proceed with the original method...
PP: [[ 0.91529369  0.        ]
 [-0.19192699  0.        ]]
QQ: [[ 0.12899709]
 [-0.01131016]]
RR: []
SS: []
NN 0.9


C:\Users\Lenovo\Anaconda3\lib\site-packages\scipy\linalg\decomp.py:60: RuntimeWarning: divide by zero encountered in true_divide
  w = (alphar + _I * alphai) / beta
C:\Users\Lenovo\Anaconda3\lib\site-packages\scipy\linalg\decomp.py:60: RuntimeWarning: invalid value encountered in true_divide
  w = (alphar + _I * alphai) / beta
C:\Users\Lenovo\Anaconda3\lib\site-packages\scipy\linalg\decomp.py:39: RuntimeWarning: invalid value encountered in greater
  m = (w.imag > 0)
C:\Users\Lenovo\Anaconda3\lib\site-packages\scipy\linalg\decomp.py:40: RuntimeWarning: invalid value encountered in less
  m[:-1] |= (w.imag[1:] < 0)  # workaround for LAPACK bug, cf. ticket #709


In [37]:
#problem 7
# generating the artificial time series
#some initialization
sigma = 0.02
num_sim = 10000
period = 250
init = (k_bar, ell_bar, Zbar)
# The simulation function 
def sim(PP, QQ, NN, init, num, t, sigma):
    '''
    ---------------------------------------------------------------
    Function to derive the simulation of variables of interest
    ---------------------------------------------------------------
    Input:
    PP: The derivative of k_t+1 against k_t  
    QQ: The derivative of k_t+1 agaisnt z_t
    NN: The derivative of z_t+1 against z_t
    RR: The derivative of l_t   against k_t
    SS: The derivative of l_t   against z_t
    init:The initial value of endogenous and 
    num:The number of simulations
    t:  The number of periods in each simulation
    sigma: the standard deviation of the productivity shock
    
    Output:
    
    '''
    # unpacking initial variables
    k_bar, ell_bar, Zbar = init
    
    # we firstly construct the epsilon matrix that goes into the shock matrix
    eps_mat = np.random.normal(0, sigma, (t + 1, num))
    
    # we then construct the shock matrix z_mat
    z_mat = np.empty((t + 1, num))
    z_mat[0, :] = np.zeros((1, num))
    for iZ in range(1, t + 1):
        z_mat[iZ, :] = (1 - NN)*Zbar + NN*z_mat[iZ-1, :] + eps_mat[iZ, :]
    
    # we will then construct the capital matrix
    k_mat = np.empty((t + 1 , num))
    k_mat[0, :] = 0
    for iK in range(1, t + 1):
        k_mat[iK, :] = PP[0,0]*k_mat[iK-1, :] + QQ[0,0]*z_mat[iK, :]
    l_mat = np.empty((t, num))
    l_mat[0:, ] = 0
    for iL in range(1, t):
        l_mat[iL, :] = PP[1,0]*l_mat[iL-1, :] + QQ[1,0]*z_mat[iL, :]
        
    kp_mat = k_bar*np.exp(k_mat[1:,])
    k_mat = k_bar*np.exp(k_mat[:-1,])
    l_mat = ell_bar*np.exp(l_mat)
    Y_mat = k_mat**alpha*(np.exp(z_mat[:-1,])*l_mat)**(1-alpha)
    w_mat = (1-alpha)*Y_mat/l_mat
    r_mat = alpha*Y_mat/k_mat
    c_mat = (w_mat*l_mat + (r_mat - delta)*k_mat)  + k_mat - kp_mat
    i_mat = kp_mat - (1 - delta)*k_mat
    
    return Y_mat, c_mat, i_mat, l_mat, k_mat, w_mat, r_mat
Y, C, I, L, K, W, R = sim(PP, QQ, NN, init, num_sim, period, sigma)
#plotting


    
    
    
    



1.2845067662323033

In [22]:
params

(0.4, 0.98, 2.5, 0.1, 0.5, 1.5, 0.9, 0.05)

In [13]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
